In [ ]:
# Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/src')

In [ ]:
# Install necessary packages from requirements.txt
req_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/requirements.txt'
with open(req_path, 'r') as file:
    requirements = file.read().splitlines()

for package in requirements:
    !pip install {package}

In [ ]:
from ragpipelines import *

# Load configuration from config.yaml
import yaml
config_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/config.yaml'
with open(config_path, 'r') as file:
    config = yaml.safe_load(file)

!pip install trulens_eval==0.33.0

In [ ]:
# Import libraries for evaluation
import pandas as pd
import numpy as np
from trulens_eval.feedback import GroundTruthAgreement
from trulens_eval import Tru, OpenAI, Feedback, TruLlama

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = config['openai_api_key']

In [ ]:
# Initialize TruLens and OpenAI provider
tru = Tru()
openai_provider = OpenAI(model_engine="gpt-4o-mini")

In [ ]:
tru.reset_database()

In [ ]:
# Load evaluation questions and answers
eval_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/eval_questions.xlsx'
qa_df = pd.read_excel(eval_path, sheet_name="Sheet1")
qa_set = [{"query": item["Question"], "response": item["Answer"]} for index, item in qa_df.iterrows()]

In [ ]:
# Define the evaluation metrics
# answer relevance
f_qa_relevance = Feedback(
    openai_provider.relevance_with_cot_reasons, name="Answer Relevance"
).on_input_output()

In [ ]:
# context relevance
f_qs_relevance = Feedback(
    openai_provider.qs_relevance_with_cot_reasons, name="Context Relevance"
).on_input().on(TruLlama.select_source_nodes().node.text).aggregate(np.mean)

In [ ]:
from trulens_eval import Select

In [ ]:
# Groundedness (sometimes called faithfulness)
f_groundedness = (
    Feedback(openai_provider.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
)

In [ ]:
# Ground truth agreement (sometimes called answer correctness)
f_groundtruth = Feedback(
    GroundTruthAgreement(qa_set).agreement_measure, name = "Answer Correctness"
).on_input_output()

In [ ]:
metrics = [f_qa_relevance, f_qs_relevance, f_groundedness, f_groundtruth]

In [ ]:
# Function to create TruLens recorder
def get_trulens_recorder(query_engine, app_id):
    return TruLlama(query_engine, feedbacks=metrics, app_id=app_id)

In [ ]:
# Base class for RAG pipelines evaluation
class RAGEvaluator:
    def __init__(self, pipeline_class, app_id, api_key, document_path, llm_model, embed_model):
        self.pipeline = pipeline_class(api_key, document_path, llm_model, embed_model)
        self.query_engine = self.pipeline.query_engine
        self.recorder = get_trulens_recorder(self.query_engine, app_id=app_id)

    def evaluate(self, qa_set):
        with self.recorder as recording:
            for q in qa_set:
                self.query_engine.query(q["query"])

In [ ]:
api_key = config['openai_api_key']
document_path = '/content/drive/MyDrive/3. MscIS 22-24/Thesis/final code files/privacy-pmi.pdf'
llm_model = config['llm_model']
embed_model = config['embed_model']

In [ ]:
# This function evaluates a given RAG pipeline by using the TruLens framework
def evaluate_pipeline(pipeline, qa_set, app_id, tru, metrics):
    # Initialize TruLens recorder for the pipeline
    recorder = TruLlama(
        app=pipeline.query_engine,  # Assuming the query_engine is the app here
        feedbacks=metrics,
        app_id=app_id
    )

    with recorder as recording:
        for q in qa_set:
            pipeline.query(q["query"])

    # Retrieve and return the records and feedback
    records, feedback = tru.get_records_and_feedback(app_ids=[app_id])
    return records, feedback

In [ ]:
# Initialize and evaluate each pipeline

basic_pipeline = BasicRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                  llm_model=config['llm_model'], embed_model=config['embed_model'])

In [ ]:
evaluate_pipeline(basic_pipeline, qa_set, "0. Basic Query Engine", tru, metrics)

In [ ]:
sentence_window_rag_without_rerank = SentenceWindowRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                                               llm_model=config['llm_model'], embed_model=config['embed_model'],
                                                               rerank=False)

In [ ]:
evaluate_pipeline(sentence_window_rag_without_rerank, qa_set, "1. Sentence Window Query Engine - W/o Reranking", tru, metrics)

In [ ]:
sentence_window_rag_with_rerank = SentenceWindowRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                                            llm_model=config['llm_model'], embed_model=config['embed_model'],
                                                            rerank=True)

In [ ]:
evaluate_pipeline(sentence_window_rag_with_rerank, qa_set, "2. Sentence Window Query Engine - w/ re-ranking", tru, metrics)

In [ ]:
automerging_pipeline = AutoMergingRAGPipeline(api_key=config['openai_api_key'], document_path=document_path,
                                              llm_model=config['llm_model'], embed_model=config['embed_model'])

In [ ]:
evaluate_pipeline(automerging_pipeline, qa_set, "3. Automerging Query Engine - w/ re-ranking", tru, metrics)

In [ ]:
automerging_pipeline_with_prompting = AutoMergingRAGPipelineWithPrompting(
    api_key=config['openai_api_key'],
    document_path=document_path,
    llm_model=config['llm_model'],
    embed_model=config['embed_model']
)

In [ ]:
evaluate_pipeline(
    automerging_pipeline_with_prompting,
    qa_set,
    "4. Auto-Merging Query Engine - w/ re-ranking and prompt engineering",
    tru,
    metrics
)

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

In [ ]:
#tru.stop_dashboard()